In [ ]:
pip install nnsatool

     |████████████████████████████████| 2.3MB 13.7MB/s 
     |████████████████████████████████| 901kB 51.6MB/s 
     |████████████████████████████████| 3.3MB 49.3MB/s 
  Created wheel for nnsatool: filename=NNSAtool-0.0.6-cp37-none-any.whl size=6349 sha256=d0b6e687aa6ca083735ca6510141fe40aa5366ee40f705383b1b33222cf99bae
  Stored in directory: /root/.cache/pip/wheels/9d/34/38/77a350cd8f79a44552d95c31396def86b32f948d068e61eb8c
Successfully built nnsatool


In [ ]:
import nnsatool as nnsa

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
data_path = "/content/gdrive/My Drive/Data/"
model_path = "/content/gdrive/My Drive/Models/"

In [ ]:
import numpy as np
import pandas as pd
import torch

In [ ]:
RANDOM_SEED = 0
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
!gdown --id 1S6qMioqPJjyBLpLVz4gmRTnJHnjitnuV
!gdown --id 1zdmewp7ayS4js4VtrJEHzAheSW-5NBZv

Downloading...
From: https://drive.google.com/uc?id=1S6qMioqPJjyBLpLVz4gmRTnJHnjitnuV
To: /content/apps.csv
100% 134k/134k [00:00<00:00, 52.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zdmewp7ayS4js4VtrJEHzAheSW-5NBZv
To: /content/reviews.csv
7.17MB [00:00, 43.9MB/s]


In [ ]:
data_frame = pd.read_csv("reviews.csv")

In [ ]:
def scr2snt(scr):
  scr = int(scr)
  if scr <= 2:
    return 0
  elif scr == 3:
    return 1
  else:
    return 2

In [ ]:
data_frame['sentiment'] = data_frame.score.apply(scr2snt)

In [ ]:
tags = ['negative', 'neutral', 'positive']

In [ ]:
BERT_MODEL_NAME = 'bert-base-cased'

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train, df_test = train_test_split(data_frame, test_size=0.1, random_state=RANDOM_SEED)

In [ ]:
tokenizer = nnsa.AutoTokenizer.from_pretrained(BERT_MODEL_NAME)
bert = nnsa.AutoModel.from_pretrained(BERT_MODEL_NAME)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
ld_test = nnsa.TokenizedDataLoaderFactory.get_instance(
    data_frame = df_test,
    src_idx = 'content',
    trg_idx = 'sentiment',
    tokenizer = tokenizer,
    max_len = 175,
    batch_size = 16
)

In [ ]:
from torch import nn

In [ ]:
class Analyzer(nn.Module):
  def __init__(self, bert, p, num_tags):
    super(Analyzer, self).__init__()
    self.inp2emb = bert
    self.drop = nn.Dropout(p=p)
    self.emb2out = nn.Linear(self.inp2emb.config.hidden_size, num_tags)

  def forward(self, input_ids, attention_mask):
    emb = self.inp2emb(input_ids=input_ids, attention_mask=attention_mask)[1]
    emb = self.drop(emb)
    out = self.emb2out(emb)
    return out

In [ ]:
analyzer = Analyzer(bert, 0.2, len(tags))
analyzer = analyzer.to(device)

In [ ]:
def eval(model, loader, crit, device, num_ex):
  model = model.eval()
  losses = []
  correct = 0

  with torch.no_grad():
    for data in loader:
      input_ids = data[nnsa.PreProcConst.INP_IDS].to(device)
      attention_mask = data[nnsa.PreProcConst.ATT_MSK].to(device)
      trg = data[nnsa.PreProcConst.TRG_ITM].to(device)

      out, pred = model((input_ids, attention_mask))
      loss = crit(out, trg)
      correct += torch.sum(pred == trg)
      losses.append(loss.item())
  
  return correct.double() / num_ex, np.mean(losses)

In [ ]:
analyzer.load_state_dict(torch.load(model_path + 'Agg_02_3e5.pt'))

<All keys matched successfully>

In [ ]:
crit = nn.CrossEntropyLoss().to(device)

In [ ]:
res = eval(analyzer, ld_test, crit, device, len(df_test))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
res[0].item()

0.8692063492063492

In [ ]:
model = nnsa.SemanticAggregator(
    embedding = nnsa.EmbeddingModule(
        basis = bert,
        mode = nnsa.EmbModeConst.AGG_OUT,
        dropout = 0.2
    ),
    transition = nnsa.TransitionModule(
        chains = [nnsa.ModuleChainFactory.get_linear_chain(
            dim_desc = [bert.config.hidden_size, len(tags)]
        )]
    )
)

In [ ]:
list(analyzer.state_dict().keys())

['inp2emb.embeddings.position_ids',
 'inp2emb.embeddings.word_embeddings.weight',
 'inp2emb.embeddings.position_embeddings.weight',
 'inp2emb.embeddings.token_type_embeddings.weight',
 'inp2emb.embeddings.LayerNorm.weight',
 'inp2emb.embeddings.LayerNorm.bias',
 'inp2emb.encoder.layer.0.attention.self.query.weight',
 'inp2emb.encoder.layer.0.attention.self.query.bias',
 'inp2emb.encoder.layer.0.attention.self.key.weight',
 'inp2emb.encoder.layer.0.attention.self.key.bias',
 'inp2emb.encoder.layer.0.attention.self.value.weight',
 'inp2emb.encoder.layer.0.attention.self.value.bias',
 'inp2emb.encoder.layer.0.attention.output.dense.weight',
 'inp2emb.encoder.layer.0.attention.output.dense.bias',
 'inp2emb.encoder.layer.0.attention.output.LayerNorm.weight',
 'inp2emb.encoder.layer.0.attention.output.LayerNorm.bias',
 'inp2emb.encoder.layer.0.intermediate.dense.weight',
 'inp2emb.encoder.layer.0.intermediate.dense.bias',
 'inp2emb.encoder.layer.0.output.dense.weight',
 'inp2emb.encoder.laye

In [ ]:
list(model.state_dict().keys())

['embedding.inp2emb.embeddings.position_ids',
 'embedding.inp2emb.embeddings.word_embeddings.weight',
 'embedding.inp2emb.embeddings.position_embeddings.weight',
 'embedding.inp2emb.embeddings.token_type_embeddings.weight',
 'embedding.inp2emb.embeddings.LayerNorm.weight',
 'embedding.inp2emb.embeddings.LayerNorm.bias',
 'embedding.inp2emb.encoder.layer.0.attention.self.query.weight',
 'embedding.inp2emb.encoder.layer.0.attention.self.query.bias',
 'embedding.inp2emb.encoder.layer.0.attention.self.key.weight',
 'embedding.inp2emb.encoder.layer.0.attention.self.key.bias',
 'embedding.inp2emb.encoder.layer.0.attention.self.value.weight',
 'embedding.inp2emb.encoder.layer.0.attention.self.value.bias',
 'embedding.inp2emb.encoder.layer.0.attention.output.dense.weight',
 'embedding.inp2emb.encoder.layer.0.attention.output.dense.bias',
 'embedding.inp2emb.encoder.layer.0.attention.output.LayerNorm.weight',
 'embedding.inp2emb.encoder.layer.0.attention.output.LayerNorm.bias',
 'embedding.inp2

In [ ]:
old_dict = torch.load(model_path + 'Agg_02_3e5.pt')

In [ ]:
from collections import OrderedDict

In [ ]:
new_dict = OrderedDict()

In [ ]:
rule = {
    'inp2emb' : nnsa.ModuleConst.EMBED + '.' + nnsa.ModuleConst.INP2EMB,
    'emb2out' : nnsa.ModuleConst.TRANS + '.' + nnsa.ModuleConst.VEC2VEC + '0_0'
}

In [ ]:
for key, val in old_dict.items():
  new_dict[rule[key[:7]] + key[7:]] = val

In [ ]:
new_dict['transition.vec2vec0_0.weight']

tensor([[-0.0152,  0.0207,  0.0095,  ..., -0.0067, -0.0163, -0.0037],
        [ 0.0222, -0.0227, -0.0194,  ..., -0.0095, -0.0131, -0.0345],
        [-0.0019,  0.0295,  0.0013,  ...,  0.0207, -0.0056,  0.0018]],
       device='cuda:0')

In [ ]:
model.load_state_dict(new_dict)

<All keys matched successfully>

In [ ]:
model = model.to(device)

In [ ]:
res = eval(model, ld_test, crit, device, len(df_test))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
res[0].item()

0.8692063492063492

JSON